# ANA680_Week_3:

### Build linear regression model

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split

## Load and inspect data

In [15]:
wine_quality = pd.read_csv("winequality.csv", header=0)
wine_quality.head()

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,6,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8
1,6,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5
2,6,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1
3,6,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9
4,6,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9


## Split into Train(80%) and Test(20%)

In [16]:
# Split features and target
X = wine_quality.iloc[:, 1:]
y = wine_quality.iloc[:, 0]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=680)

## <u>Linear regression WITHOUT container technology</u>

In [17]:
# Set up environment
import numpy as np
import sagemaker
from sagemaker import get_execution_role, Session
from sagemaker.inputs import TrainingInput
from sagemaker.amazon.linear_learner import LinearLearner

## Shape data for Sagemaker

In [18]:
train_array = np.concatenate([y_train.to_numpy().reshape(-1, 1), X_train.to_numpy()], axis=1)
test_array = np.concatenate([y_test.to_numpy().reshape(-1, 1), X_test.to_numpy()], axis=1)

np.savetxt("train_split.csv", train_array, delimiter=",")
np.savetxt("test_split.csv", test_array, delimiter=",")

## Start session

In [19]:
# Start SageMaker session
role = get_execution_role()
session = Session()
bucket = "sagemaker-us-east-1-421498156986"
prefix = "linear-regression-wine"

# Upload train and test splits to S3
s3_train_path = f"s3://{bucket}/{prefix}/train_split.csv"
s3_test_path = f"s3://{bucket}/{prefix}/test_split.csv"

print(s3_train_path)
print(s3_test_path)

s3://sagemaker-us-east-1-421498156986/linear-regression-wine/train_split.csv
s3://sagemaker-us-east-1-421498156986/linear-regression-wine/test_split.csv


## Define Training objects

In [20]:
train_input = TrainingInput(s3_data=s3_train_path, content_type="text/csv")
test_input = TrainingInput(s3_data=s3_test_path, content_type="text/csv")
print(type(train_input))
print(type(test_input))

<class 'sagemaker.inputs.TrainingInput'>
<class 'sagemaker.inputs.TrainingInput'>


In [21]:
# Train model
linear = LinearLearner(
    role=role,
    instance_count=1,
    instance_type='ml.m3.medium',
    predictor_type='regressor',
    output_path=f's3://{bucket}/linear-regression-output')

linear.fit('s3://sagemaker-us-east-1-421498156986/linear-regression-wine/train_split.csv')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 │   predictor_type='regressor',                                                             │
│    7 │   output_path=f's3://{bucket}/linear-regression-output')                                  │
│    8                                                                                             │
│ ❱  9 linear.fit('s3://sagemaker-us-east-1-421498156986/linear-regression-wine/train_split.csv    │
│   10                                                                                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/pipeline_c │
│ ontext.py:346 in wrapper                                                                         │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/amazon/amazon_estim │
│ ator.py:263 in fit                                                                               │
│                                                                                                  │
│   260 │   │   │   │   will be unassociated.                                                      │
│   261 │   │   │   │   * `TrialComponentDisplayName` is used for display in Studio.               │
│   262 │   │   """                                                                                │
│ ❱ 263 │   │   self._prepare_for_training(records, job_name=job_name, mini_batch_size=mini_batc   │
│   264 │   │                                                                                      │
│   265 │   │   experiment_config = check_and_get_run_experiment_config(experiment_config)         │
│   266 │   │   self.latest_training_job = _TrainingJob.start_new(                                 │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/amazon/linear_learn │
│ er.py:438 in _prepare_for_training                                                               │
│                                                                                                  │
│   435 │   │   │   if num_records is None:                                                        │
│   436 │   │   │   │   raise ValueError("Must provide train channel.")                            │
│   437 │   │   else:                                                                              │
│ ❱ 438 │   │   │   num_records = records.num_records                                              │
│   439 │   │                                                                                      │
│   440 │   │   # mini_batch_size can't be greater than numbe

In [ ]:
# Deploy model
predictor = linear.deploy(instance_type='ml.m3.medium', 
                          initial_instance_count=1, serializer = CSVSerializer())


In [ ]:
# Run prediction
result = predictor.predict(X_test.to_numpy())
print(result)


In [ ]:
# Clean up by deleting endpoint
sagemaker.Session().delete_endpoint(predictor.endpoint_name)

In [ ]:
print(train_df.isnull().sum())

## <u>Linear regression WITH containers (from Docker)</u>

# END